In [1]:
import os

In [2]:
os.environ['MEDLEYDB_PATH'] = '/home/bmcfee/data/MedleyDB/'

In [3]:
import jams
import jams.display
import matplotlib.pyplot as plt
%matplotlib nbagg

In [4]:
from IPython.display import Audio

In [5]:
import numpy as np

In [6]:
import medleydb
import medleydb.mix

In [18]:
from joblib import Parallel, delayed

In [67]:
def encode_intervals(times, values, threshold=0.5):
    
    # times is an array of frame times
    # values are the observed activations per frame
    
    # Threshold the activations
    v_t = values >= threshold
    
    on = False
    
    for t, v in zip(times, v_t):
        if v and not on:
            on = v
            start = t
        elif on and not v:
            on = False
            yield dict(time=start, duration=t-start)
    if on:
        yield dict(time=start, duration=t-start)
    
def jam_out(mtrack, output_path, stem_indices=None):
    
    if stem_indices is None:
        stem_indices = list(mtrack.stems.keys())
        
    meta = jams.FileMetadata(title=mtrack.title, artist=mtrack.artist, duration=mtrack.duration)
    jam = jams.JAMS(file_metadata=meta)
    ann = jams.Annotation('tag_medleydb_instruments', duration=mtrack.duration)
    
    time_act = np.asarray(mtrack.stem_activations).T
    
    time = time_act[0]
    
    for stem in stem_indices:
        tag = mtrack.stems[stem].instrument
        if tag not in jams.schema.values(ann.namespace):
            continue
        
        if stem not in mtrack.stem_activations_idx:
            continue
            
        row = mtrack.stem_activations_idx[stem]
        
        for obs in encode_intervals(time, time_act[row], threshold=0.5):
            ann.append(time=obs['time'], duration=obs['duration'], value=tag, confidence=None)
    
    jam.annotations.append(ann)
    
    jam.save(output_path)

In [78]:
def mix_out(mtrack, instruments, output_prefix, ext='ogg', redo=True):
    """Remixes a multitrack with anything type of vocals removed.
    If no vocals are present, the mix will be a simple weighted linear remix.

    Parameters
    ----------
    mtrack : Multitrack
        Multitrack object
    output_prefix : str
        Prefix to save output file.

    Returns
    -------
    stem_indices : list
        List of stem indices used in mix.

    """
    
    # What instrumenst are in this mix?
    mix_instruments = set([stem.instrument for stem in mtrack.stems.values()])
    
    
    # For each instrument we care about modeling,
    # generate a mix with that instrument removed
    loop_set = mix_instruments & set(instruments)

    # We'll throw in a null-subtraction just to keep a full mix in with the rest
    for inst in loop_set | set(['']):
        
        stem_indices = [idx for idx in mtrack.stems if mtrack.stems[idx].instrument != inst]

        # If there's nothing left, we're done
        if not stem_indices:
            break
            
        # Make the multitrack output
        output_path = '{}---{}{}{}'.format(output_prefix, inst, os.path.extsep, ext)
        if redo or not os.path.exists(output_path):
            medleydb.mix.mix_multitrack(mtrack, output_path, stem_indices=stem_indices)
    
        # Make the annotation
        output_path = '{}---{}{}{}'.format(output_prefix, inst, os.path.extsep, 'jams')
        jam_out(mtrack, output_path, stem_indices=stem_indices)

In [58]:
def tmo(track_id, instruments, outpath, redo=False):
    mt = medleydb.MultiTrack(track_id)
    mix_out(mt, instruments, '{}/{}'.format(outpath, mt.track_id), redo=redo)

In [59]:
# The top-14 instruments, except for synth
INSTRUMENTS = [ 'drum set',
                'electric bass',
                'piano',
                'male singer',
                'clean electric guitar',
                'vocalists',
                'female singer',
                'acoustic guitar',
                'distorted electric guitar',
                'auxiliary percussion',
                'double bass',
                'violin',
                'cello',
                'flute']


In [79]:
Parallel(verbose=1, n_jobs=23)(delayed(tmo)(track_id, INSTRUMENTS, '/home/bmcfee/working/Medleydb_Downmix/')
                               for track_id in medleydb.TRACK_LIST)

[Parallel(n_jobs=23)]: Done   4 tasks      | elapsed:    4.3s
[Parallel(n_jobs=23)]: Done 122 out of 122 | elapsed:  1.4min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]